# Projectile Range
Suppose we have a projectile with which we want to hit some range. The formula for the range of a projectile is $R = \frac{{v_0}^2 \sin(2\theta_0)}{g}$ and it shows that the maximum range depends on the square of initial velocity and the angle of launch. Let's assume that we don't know the phyiscs and we want to use regression to predict the maximum range. In our case, if the angle is between 50 and 60 degrees, we see maximum range and we get a maximum target 'y'. Similarly if the velocity is is between 90 and 100 arnitrary units, we observe the maximum range.

The code cell right below creates the toy dataset you have to use to train and then test a regularized logistic regression model. Don't worry if you don't understand some parts of the code, it's using more advanced tools such as *pandas* and *scikit-learn*. Simply run the cell to obtain the "x_train, x_test, y_train, y_test" numpy arrays.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

# Set a seed for reproducibility
np.random.seed(0)

# Number of experiments
n = 15000

# Generate random angles between 10 and 90 degrees
angles = np.random.uniform(low=10, high=90, size=n)

# Generate random initial velocities
velocities = np.random.uniform(low=10, high=100, size=n)

# Combine angles and velocities into a single 2D array (this will be our input features)
X = np.column_stack((angles, velocities))

# Generate target variable
# For simplicity, let's say a hit is when angle is between 50 and 60 degrees and velocity is between 90 and 100 units
y = [1 if (50 <= angle <= 60) and (90 <= velocity <= 100) else 0 for angle, velocity in zip(angles, velocities)]

# Convert y to a numpy array
y = np.array(y)

# Balance the data sample: retain an equal number of hit and miss outcomes
import pandas as pd
df = pd.DataFrame(X, columns=['angle', 'velocity'])
df['y'] = y
df_sig = df[df['y']==1]
df_back = df[df['y']==0].sample(n=df_sig.shape[0])
df_balanced = pd.concat([df_sig, df_back])

# Convert 'angle' and 'velocity' columns back to a 2D numpy array for X
X = df_balanced[['angle', 'velocity']].values

# Convert 'y' column back to a 1D numpy array for y
y = df_balanced['y'].values

# the code below prepares training and testing datasets for you (we will go over the sklearn package in the afternoon session)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=324, stratify=y)

In the cell below, define the cost function for a regularized logistic regression model that you will use to predict wether a given projectile hits the range or not. You should pass the function a regulaz

In [2]:
### Total: 40 points
def sigmoid(z):
    z = np.clip( z, -500, 500 )           
    g = 1.0/(1.0+np.exp(-z))  ### correct sigmoid computation: 2/40

    return g

def compute_cost_logistic(X, y, w, b, lambda_ = 1): ### correct arguments: 5/40 (1 for each argument)
    """
    Computes the cost over all examples
    Args:
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
      lambda_ (scalar): Controls amount of regularization
    Returns:
      total_cost (scalar):  cost 
    """

    m,n  = X.shape ### correct initialization: 4/40
    cost = 0. ### correct initialization: 2/40
    for i in range(m): ### loop for in proper range: 4/40
        z_i = np.dot(X[i], w) + b                               
        f_wb_i = sigmoid(z_i) ### correct z_i and sigmoid call: 4/40                                           
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i) ### correct computation: 4/40     
             
    cost = cost/m ### correct computation: 2/40                                                     

    reg_cost = 0 ### correct initialization: 2/40
    for j in range(n): # ##loop for in proper range: 4/40      
        reg_cost += (w[j]**2) ### correct sum: 2/40                                         
    reg_cost = (lambda_/(2*m)) * reg_cost ### correct computation: 3/40                             
    
    total_cost = cost + reg_cost                                  
    return total_cost ### return of regularized cost: 2/40   

In the cell below, define a function that computes the gradient for the logistic regression.

In [3]:
### Total: 40 points
def compute_gradient_logistic(X, y, w, b): ### correct arguments: 4/40 (1 for each argument)
    """
    Computes the gradient for logistic regression 
 
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
    Returns
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar)      : The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape ### correct initialization: 4/40
    dj_dw = np.zeros((n,)) ### correct initialization: 4/40
    dj_db = 0. ### correct initialization: 2/40

    for i in range(m): ### loop for in proper range: 4/40   
        f_wb_i = sigmoid(np.dot(X[i],w) + b) ### correct computation: 4/40         
        err_i  = f_wb_i  - y[i] ### correct computation: 2/40                       
        for j in range(n): ### loop for in proper range: 4/40   
            dj_dw[j] = dj_dw[j] + err_i * X[i,j] ### correct computation: 4/40     
        dj_db = dj_db + err_i ### correct computation: 2/40 
    dj_dw = dj_dw/m  ### correct computation: 2/40                                  
    dj_db = dj_db/m  ### correct computation: 2/40                                  
        
    return dj_db, dj_dw ### correct return: 2/40 

In the cell below, define a function that implements the gradient descent for the logistic regression model above. **Hint**: to check wether the algorithm converges, you may want to print the cost funtion value every few iterations.

In [4]:
### Total: 40 points
import copy, math
def gradient_descent(X, y, w_in, b_in, alpha, num_iters, lambda_): ### correct arguments: 7/40 (1 for each argument)  
    """
    Performs batch gradient descent
    
    Args:
      X (ndarray (m,n)   : Data, m examples with n features
      y (ndarray (m,))   : target values
      w_in (ndarray (n,)): Initial values of model parameters  
      b_in (scalar)      : Initial values of model parameter
      alpha (float)      : Learning rate
      num_iters (scalar) : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,))   : Updated values of parameters
      b (scalar)         : Updated value of parameter 
    """
 
    J_history = [] ### correct initialization: 2/40
    w = copy.deepcopy(w_in)  ### correct copy and/or initialization: 2/40
    b = b_in ### correct initialization: 2/40
    
    for i in range(num_iters): ### loop for in proper range: 4/40   
        # Calculate the gradient and update the parameters
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b) ### correct function call: 6/40 (1 for each argument, 1 for each output)     

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw ### correct computation: 4/40                 
        b = b - alpha * dj_db ### correct computation: 4/40                 
      
        # Save cost J at each iteration
        if i<100000:      
            J_history.append( compute_cost_logistic(X, y, w, b, lambda_) ) ### call of cost function: 5/40 (1 for each argument, appending to history is optional)   

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")  ### printing cost function at intervals (nedded to check convergence) 2/40
        
    return w, b, J_history ### returning w and b (history is optional): 2/40 

In the cell below, run the gradient descent for at least 10000 iterations to train the model. Use a regularization parameter lambda_ = 1.0, and a learning rate with a given value that gives you convergence. You may have to try a few times until you reach convergence. **Hint**: don't use a learning rate value smaller than $10^{-4}$, otherwise the convergence may become too slow with this dataset.

In [5]:
### Total: 15 points
w_tmp  = np.zeros_like(x_train[0])
b_tmp  = 0.
alpha_ = 0.001 ### good value for convergence: 6/15
lambda_ = 1.0 
iters = 10000

w_out, b_out, descent_history = gradient_descent(x_train, y_train, w_tmp, b_tmp, alpha_, iters, lambda_) ### correct call: 9/15 (1 for each argument, 1 for w_out, 1 for b_out)
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

Iteration    0: Cost 0.673978824643189   
Iteration 1000: Cost 0.5283889039149194   
Iteration 2000: Cost 0.5247808774233831   
Iteration 3000: Cost 0.5212429145877682   
Iteration 4000: Cost 0.5177738308900711   
Iteration 5000: Cost 0.5143724324175792   
Iteration 6000: Cost 0.5110375181872994   
Iteration 7000: Cost 0.5077678823792483   
Iteration 8000: Cost 0.5045623164728366   
Iteration 9000: Cost 0.501419611281661   

updated parameters: w:[-0.04573487  0.04296165], b:-0.5808417584099086


In the cell below, use your model to predict wether each of the entries in the 'x_test' array hits the target. You may then print the target values 'y_test' to see how well the model predicts the outcomes.

In [6]:
### Total: 15 points
print(sigmoid(np.dot(x_test,w_out) + b_out)) ### correct: 13/15 points (dot product correct: 5/15, they add b_out 3/15, they actually call and print sigmoid 5/15)
print(y_test) ### they print y_test as suggested: 2/12 points 

[0.75463886 0.24935501 0.7450033  0.7484467  0.21515318]
[1 0 1 1 0]
